In [9]:
!pip install -U 'scikit-learn<0.24'
!pip install sklearn-crfsuite

# YOU NEED TO RESTART THE RUNTIME!!!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 4.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 4.8 MB/s 


In [7]:
# Run this cell to mount your drive to this notebook in order to read the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import re
import json
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Read Dataset

In [18]:
# Put the folder path where the datasets are located
PATH = "/content/drive/MyDrive/445/dataset"


In [19]:
# function to read data, return list of tuples each tuple represents a token contains word, pos tag, chunk tag, and ner tag
# Loading the libraries

def read_data(filename) -> list:
  file1 = open('/content/drive/MyDrive/445/dataset/'+filename, 'r')
  count = 0
  all_tuples=[]
  big_tuples=[]
  for line in file1:
      count += 1
      words=line.split()
      if len(words) !=0:
        word=words[0]
        pos_tag=words[1]
        chunk_tag=words[2]
        ner_tag=words[3]
        ###ignore docstart add here
        if word!='-DOCSTART-':
          my_tuple = (word, pos_tag, chunk_tag,ner_tag)
          all_tuples.append(my_tuple)
        else:
            next(file1,None) # consume next line
            continue # skip this line
      else:
        #print("neden", word)
        big_tuples.append(all_tuples)
        all_tuples=[]

    
  #print(words)
  file1.close()  
  return big_tuples
  ### add sentence wise 

####SOURCE: https://www.geeksforgeeks.org/read-a-file-line-by-line-in-python/
## https://stackoverflow.com/questions/2592798/how-can-i-skip-the-current-item-and-the-next-in-a-python-loop

In [20]:
# read data with your custom function
train_data = read_data("train.txt")
val_data = read_data("valid.txt")
test_data = read_data("test.txt")

In [21]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

14041
3250
3453


In [ ]:
'''
new_train=[]
for i in range(len(train_data)):
  if train_data[i]!=[]:
    new_train.append(train_data[i])

new_val=[]
for i in range(len(val_data)):
  if val_data[i]!=[]:
    new_val.append(val_data[i])

new_test=[]
for i in range(len(test_data)):
  if test_data[i]!=[]:
    new_test.append(test_data[i])


train_data=new_train
val_data=new_val
test_data=new_test
'''

'\nnew_train=[]\nfor i in range(len(train_data)):\n  if train_data[i]!=[]:\n    new_train.append(train_data[i])\n\nnew_val=[]\nfor i in range(len(val_data)):\n  if val_data[i]!=[]:\n    new_val.append(val_data[i])\n\nnew_test=[]\nfor i in range(len(test_data)):\n  if test_data[i]!=[]:\n    new_test.append(test_data[i])\n\n\ntrain_data=new_train\nval_data=new_val\ntest_data=new_test\n'

In [22]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

14041
3250
3453


# Create Gazetteer

In [ ]:
%cd /content/drive/MyDrive/445/wikipedia_pages/

/content/drive/.shortcut-targets-by-id/1VmIrQyBeIHmpSxhvMjsaRU5z3stW3pqS/wikipedia_pages


In [ ]:
os.chdir("/content/drive/MyDrive/445/wikipedia_pages/")
listt=os.listdir()

In [ ]:
'''
text=json_dictionary["text"]
start=text.find('"&gt;')
end=text.find(';/a&gt')
text[start+5:end-3]
'''

'\ntext=json_dictionary["text"]\nstart=text.find(\'"&gt;\')\nend=text.find(\';/a&gt\')\ntext[start+5:end-3]\n'

In [ ]:

# load wikipedia pages
gazetteer=[]
for i in range(len(listt)):
  path="/content/drive/MyDrive/445/wikipedia_pages/"+listt[i]
  myfile = open(path)
  json_dictionary = json.load(myfile)
  #print(json_dictionary)
  text=json_dictionary["text"]
  title=json_dictionary["title"]
  #start=text.find('"&gt;')
  #end=text.find('/a&gt')
  regex = '"&gt;([^&]*)'
  matches=re.findall(regex,text)
  if title not in gazetteer:
    gazetteer.append(title)
  #print(matches[2])
  for i in matches:
    if i not in gazetteer:
      gazetteer.append(i)
    else:
      continue
    
  

  myfile.close()

#https://www.w3schools.com/python/python_regex.asp


In [ ]:
#gazetteer

In [ ]:
# load wikipedia pages

'''
import json
file = open("/content/drive/MyDrive/445/wikipedia_pages/25.json")
json_dictionary = json.load(file)
file.close()
json_dictionary

####hrefleri sırayla alarak listeye at unique sekilde 
'''

'\nimport json\nfile = open("/content/drive/MyDrive/445/wikipedia_pages/25.json")\njson_dictionary = json.load(file)\nfile.close()\njson_dictionary\n\n####hrefleri sırayla alarak listeye at unique sekilde \n'

In [ ]:
# you can also define rules to improve your gazetteer

In [ ]:
# print the size of your gazetteer
len(gazetteer)

379408

# Models

## Conditional Random Fields (CRF)

### Extract features for CRF

In [ ]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

import sklearn_crfsuite
from sklearn.metrics import make_scorer
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
word='HEY -  3'
True if '-' in word and word.isupper() and any(char.isdigit() for char in word)  else False

True

In [ ]:
def wordshape(text):
  import re
  t1 = re.sub('[A-Z]', 'X',text)
  t2 = re.sub('[a-z]', 'x', t1)
  return re.sub('[0-9]', 'd', t2)

def short_word_shape(word_shape_text):
  string=word_shape_text[0]
  for i in range(1,len(word_shape_text)):
    if word_shape_text[i]!=string[-1]:
      string=string+word_shape_text[i]

  return string

In [ ]:
text="O'xoFnll9"
a=wordshape(text)

In [ ]:
short_word_shape(a)

"X'xXxd"

In [ ]:
a

"X'xxXxxxd"

In [ ]:
# create a function to extract features for each token
ps = PorterStemmer()
stop_list=stopwords.words("english")



def token2features(sentence: list, idx: int) -> dict:
    word = sentence[idx][0]
    postag = sentence[idx][1]
    chunktag = sentence[idx][2]
    neighbors=[]
    #nertag = sentence[i][3] this is y label 
    w_shape=wordshape(word)
    short_w_shape=short_word_shape(w_shape)
    stem= ps.stem(word)
    features = {
        'stem': stem,
        'pos_tag':postag,
        'chunk_tag':chunktag,
        'SOS': False, 
        'EOS':False, 
        'start_capital': word[0].isupper(),
        'w_shape':w_shape ,
        'w_short_shape':short_w_shape,
        'contain_num':  any(char.isdigit() for char in word),
        'contain_hyphen': True if '-' in word else False,
        'upper_digit_dash':True if '-' in word and word.isupper() and any(char.isdigit() for char in word)  else False, ### burada ilk letter mı hepsi mi upper
        'contain_prefix': word[:4],
        'contain_suffix':word[-4:], 
        'uppercase':word.isupper(),
        'is_stopword': True  if word in stop_list else False,
        'neighboring words': [], ##first empty?
        'short_shape_neighbor':"",
        'shape_neighbor':"", 
        'is_gazetteer': True  if word in gazetteer else False,
       }

    if idx > 0:  #previous neighbor
      word_prev = sentence[idx-1][0] 
      postag_prev = sentence[idx-1][1]
      chunktag_prev = sentence[idx-1][2]
      word_shape=wordshape(word_prev)
      short_w_shape=short_word_shape(word_shape)
      neighbors.append(word_prev)
      features.update({
          'neighboring words': neighbors,
          'short_shape_neighbor':short_w_shape, 
          'shape_neighbor':word_shape,  
      })
    else:
      features['SOS'] = True

    if idx < len(sentence)-1: ###next neighbor 
      word_next = sentence[idx+1][0]
      postag_next = sentence[idx+1][1]
      chunktag_next = sentence[idx+1][2]
      word_shape=wordshape(word_next)
      short_w_shape=short_word_shape(word_shape)
      neighbors.append(word_next)
      features.update({
          'neighboring words': neighbors,
          'short_shape_neighbor':short_w_shape, 
          'shape_neighbor':word_shape,  
      })
    else:
        features['EOS'] = True
    return features

####SOURCE : https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html
#### https://stackoverflow.com/questions/19859282/check-if-a-string-contains-a-number
##https://stackoverflow.com/questions/49945812/is-there-any-word-shape-feature-library-for-ner-in-python

In [ ]:
# define function to process each token given a sentence
def sent2features(sentence: list) -> list:
  return [token2features(sentence, i) for i in range(len(sentence))]

# get named entity labels from the sentence
def sent2labels(sentence: list) -> list:
  return [label for token, postag, chunktag ,label in sentence]

####SOURCE : https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html

In [ ]:
# prepare inputs and labels

train_sents = [sent2features(s) for s in train_data]
val_sents = [sent2features(s) for s in val_data]
test_sents = [sent2features(s) for s in test_data]

train_labels = [sent2labels(s) for s in train_data]
val_labels = [sent2labels(s) for s in val_data]
test_labels = [sent2labels(s) for s in test_data]


In [ ]:
len(train_sents)

14041

In [ ]:
train_sents[1][0] ###num of features

{'EOS': False,
 'SOS': True,
 'chunk_tag': 'B-NP',
 'contain_hyphen': False,
 'contain_num': False,
 'contain_prefix': 'Pete',
 'contain_suffix': 'eter',
 'is_gazetteer': True,
 'is_stopword': False,
 'neighboring words': ['Blackburn'],
 'pos_tag': 'NNP',
 'shape_neighbor': 'Xxxxxxxxx',
 'short_shape_neighbor': 'Xx',
 'start_capital': True,
 'stem': 'peter',
 'upper_digit_dash': False,
 'uppercase': False,
 'w_shape': 'Xxxxx',
 'w_short_shape': 'Xx'}

In [ ]:
import nltk
import sklearn_crfsuite
from sklearn.model_selection import GridSearchCV

# Set the hyperparameter space that will be scanned.
params = {'algorithm':['l2sgd','lbfgs'] , 'all_possible_transitions':[True,False],'max_iterations':[20,100]}

# initialize GridSearchCV for CRF

crf = sklearn_crfsuite.CRF()

crf_grid_search = GridSearchCV(crf, param_grid=params, cv=5)


# fitting the model for grid search 
crf_grid_search.fit(train_sents , train_labels, X_dev=val_sents, y_dev=val_labels)

# print best parameter after tuning 

print("Best parameters for crf",crf_grid_search.best_params_)

### https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#hyperparameter-optimization

Best parameters for crf {'algorithm': 'lbfgs', 'all_possible_transitions': True, 'max_iterations': 100}


In [ ]:
# initialize and train a crf model with best hyper-parameters
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(train_sents, train_labels,X_dev=val_sents, y_dev=val_labels)
labels = list(crf.classes_)
labels.remove('O')

y_pred = crf.predict(test_sents)


In [ ]:
# calculate f1-score and classification report for test using sklearn_crfsuite.metrics class

######## ADDD THIS
f1=metrics.flat_f1_score(test_labels, y_pred,
                      average='weighted', labels=labels)
report=metrics.flat_classification_report(test_labels, y_pred)
print(report)
print("F1 score: ", f1)

              precision    recall  f1-score   support

       B-LOC       0.83      0.88      0.86      1668
      B-MISC       0.83      0.76      0.79       702
       B-ORG       0.81      0.74      0.77      1661
       B-PER       0.86      0.85      0.85      1617
       I-LOC       0.73      0.75      0.74       257
      I-MISC       0.62      0.65      0.64       216
       I-ORG       0.72      0.77      0.74       835
       I-PER       0.88      0.95      0.92      1156
           O       0.99      0.99      0.99     38323

    accuracy                           0.96     46435
   macro avg       0.81      0.82      0.81     46435
weighted avg       0.96      0.96      0.96     46435

F1 score:  0.8205838390809748


In [ ]:
#train_sents[:5]

In [ ]:
keys=[]
for i in train_sents[0][1].keys():
  keys.append(i)
#keys

In [ ]:
# start from the stem of the token and add features one by one and train a new model with each feature that you add

text_file = open("/content/drive/MyDrive/445/crf_feature_metrics.txt", "w")
f1_arr=[]
features=[]
pre_arr=[]
recall_arr=[]
preds=[]
for k in range(1,20):
  new_train=[]
  for i in range(len(train_sents)):
    new_sent=[]
    for j in range(len(train_sents[i])):
      res = {key: train_sents[i][j][key] for key in keys[:k]}
      new_sent.append(res)

    new_train.append(new_sent)

  crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True)
  
  crf.fit(new_train, train_labels)
  labels = list(crf.classes_)
  labels.remove('O')

  y_pred = crf.predict(val_sents)
  f1=metrics.flat_f1_score(val_labels, y_pred,
                        average='weighted', labels=labels)
  precision=metrics.flat_precision_score(val_labels, y_pred,
                        average='weighted', labels=labels)
  recall=metrics.flat_recall_score(val_labels, y_pred,
                        average='weighted', labels=labels)
  features.append('+' + str(keys[:k][-1]))
  pre_arr.append(precision)
  recall_arr.append(recall)
  f1_arr.append(f1)
  preds.append(y_pred)
  
  # Open a file with access mode 'a'
  with open("/content/drive/MyDrive/445/crf_feature_metrics.txt", "a") as file_object:
      # Append 'hello' at the end of file
      file_object.write(str(keys[:k])+' '+'f1: '+str(f1)+' '+'pre: '+ str(precision)+' '+'re: '+str(recall)+'\n')
    

In [ ]:
# display the result table
d = {'Features': features, 'F1': f1_arr, 'Precision':pre_arr, 'Recall':recall_arr}
df = pd.DataFrame(data=d)
df

,Features,F1,Precision,Recall
0,+stem,0.621541,0.805082,0.510868
1,+pos_tag,0.759779,0.783793,0.748576
2,+chunk_tag,0.749769,0.777778,0.731838
3,+SOS,0.761789,0.790533,0.743113
4,+EOS,0.765415,0.792304,0.748227
5,+start_capital,0.780486,0.786615,0.781820
6,+w_shape,0.792287,0.796922,0.789957
7,+w_short_shape,0.788833,0.793605,0.786702
8,+contain_num,0.787757,0.794538,0.784378
9,+contain_hyphen,0.782122,0.785836,0.781355


In [ ]:
# display the classification report for the best model
crf.fit(train_sents, train_labels)
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(test_sents)
print(metrics.flat_classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

       B-LOC       0.83      0.88      0.86      1668
      B-MISC       0.83      0.76      0.79       702
       B-ORG       0.81      0.74      0.77      1661
       B-PER       0.86      0.85      0.85      1617
       I-LOC       0.73      0.75      0.74       257
      I-MISC       0.62      0.65      0.64       216
       I-ORG       0.72      0.77      0.74       835
       I-PER       0.88      0.95      0.92      1156
           O       0.99      0.99      0.99     38323

    accuracy                           0.96     46435
   macro avg       0.81      0.82      0.81     46435
weighted avg       0.96      0.96      0.96     46435



## Recurrent Neural Network (RNN)

In [10]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import Dense, Flatten, Embedding, Input, Dropout, LSTM, TimeDistributed, Bidirectional
from keras.callbacks import ModelCheckpoint

from gensim.models import Word2Vec
import gensim.downloader as api

In [11]:
!pip install seqeval
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
# find unique labels and create dictionary to map each label to a unique integer value

label_dict={}
for i in train_labels:
  #print(i)
  for j in i:
    if j not in label_dict:
      label_dict[j]=0
k=0
for i in label_dict.keys():

  label_dict[i]=k
  k=k+1


In [14]:
label_dict

{'B-LOC': 5,
 'B-MISC': 2,
 'B-ORG': 0,
 'B-PER': 3,
 'I-LOC': 8,
 'I-MISC': 7,
 'I-ORG': 6,
 'I-PER': 4,
 'O': 1}

In [23]:
train_data[0]

[('EU', 'NNP', 'B-NP', 'B-ORG'),
 ('rejects', 'VBZ', 'B-VP', 'O'),
 ('German', 'JJ', 'B-NP', 'B-MISC'),
 ('call', 'NN', 'I-NP', 'O'),
 ('to', 'TO', 'B-VP', 'O'),
 ('boycott', 'VB', 'I-VP', 'O'),
 ('British', 'JJ', 'B-NP', 'B-MISC'),
 ('lamb', 'NN', 'I-NP', 'O'),
 ('.', '.', 'O', 'O')]

In [24]:
train_labels[0]

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [25]:
def sent2tokens(sentence: list) -> list:
  return [token for token, postag, chunktag ,label in sentence]

def labels2integers(labels: list) -> list:
  return [label_dict[label] for label in labels]

train_tokens = [sent2tokens(s) for s in train_data]
val_tokens = [sent2tokens(s) for s in val_data]
test_tokens = [sent2tokens(s) for s in test_data]

train_labels_int = [labels2integers(s) for s in train_labels]
val_labels_int = [labels2integers(s) for s in val_labels]
test_labels_int = [labels2integers(s) for s in test_labels]


In [26]:
train_labels_int[:3]

[[0, 1, 2, 1, 1, 1, 2, 1, 1], [3, 4], [5, 1]]

In [27]:
train_labels_int[1]

[3, 4]

In [28]:
# Create your own word embeddings from scratch and load a pretrained word embeddings

import gensim.downloader as api

model_gensim = Word2Vec(sentences=train_tokens,min_count=1,size=50) 


glove_vectors = api.load('glove-twitter-50')



#all pre-trained model options below
'''
['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']
'''


# You can check https://radimrehurek.com/gensim/models/word2vec.html for training a word embeddings from scratch

# You can check https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html and https://github.com/RaRe-Technologies/gensim-data for loading pretrained word embeddings. 


[==================================================] 100.0% 199.5/199.5MB downloaded


"\n['fasttext-wiki-news-subwords-300',\n 'conceptnet-numberbatch-17-06-300',\n 'word2vec-ruscorpora-300',\n 'word2vec-google-news-300',\n 'glove-wiki-gigaword-50',\n 'glove-wiki-gigaword-100',\n 'glove-wiki-gigaword-200',\n 'glove-wiki-gigaword-300',\n 'glove-twitter-25',\n 'glove-twitter-50',\n 'glove-twitter-100',\n 'glove-twitter-200',\n '__testing_word2vec-matrix-synopsis']\n"

In [29]:
# preprare your dataset for RNN classifier (you need to add padding to labels as well)
# Prepare your dataset for CNN classifier
my_tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=1000000000000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,)

max_tokens=0
for j in test_tokens:
  length=len(j)
  if length>max_tokens:
   max_tokens=length


print( max_tokens) #### set for max padding

max_pad=max_tokens ### 124
my_tokenizer.fit_on_texts(train_tokens)
train_tokens2 = my_tokenizer.texts_to_sequences(train_tokens) 
val_tokens2 = my_tokenizer.texts_to_sequences(val_tokens) 
test_tokens2 = my_tokenizer.texts_to_sequences(test_tokens) 

#train_labels = my_tokenizer.texts_to_sequences(train_labels) 

#####PADDING 
train_tokens2 = pad_sequences(train_tokens2, padding='post', maxlen=max_pad)
train_labels_int = pad_sequences(train_labels_int, padding='post', maxlen=max_pad,value=1)

val_tokens2 = pad_sequences(val_tokens2, padding='post', maxlen=max_pad)
val_labels_int = pad_sequences(val_labels_int, padding='post', maxlen=max_pad,value=1)


test_tokens2 = pad_sequences(test_tokens2, padding='post', maxlen=max_pad)
test_labels_int = pad_sequences(test_labels_int, padding='post', maxlen=max_pad,value=1)



124


In [30]:
len(train_labels_int[0])

124

In [31]:
np.unique(val_labels_int)

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32)

In [32]:
train_tokens2

array([[  988, 10950,   204, ...,     0,     0,     0],
       [  773,  1871,     0, ...,     0,     0,     0],
       [  725,   149,     0, ...,     0,     0,     0],
       ...,
       [ 3093,    20,  2146, ...,     0,     0,     0],
       [   88,    84,     0, ...,     0,     0,     0],
       [ 4383,    16,  2534, ...,     0,     0,     0]], dtype=int32)

In [33]:
train_labels_int

array([[0, 1, 2, ..., 1, 1, 1],
       [3, 4, 1, ..., 1, 1, 1],
       [5, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1]], dtype=int32)

In [111]:
# Create Embedding Matrices and Layers

from numpy import array,asarray,zeros

vocab_size=len(my_tokenizer.word_index)+1

pretrained=False

def create_embedding_matrix(pretrained, my_tokenizer ):
  if pretrained: #####PRE-TRAINED WORD EMBEDDING MATRIX
    pretrained=True
    embedding_matrix = zeros((vocab_size, 50))
    for word, index in my_tokenizer.word_index.items():
      try:
        embedding_vector = glove_vectors[word]
        #print(embedding_vector)
        embedding_matrix[index] = embedding_vector
        #print("okpre")
      except:
        continue
  else: #####TRAINED FROM SCRATCH WORD EMBEDDING MATRIX
    pretrained=False
    embedding_matrix = zeros((vocab_size, 50))
    for word, index in my_tokenizer.word_index.items():
      try:
        embedding_vector = model_gensim[word]
        #print(embedding_vector)
        embedding_matrix[index] = embedding_vector
        #print("ok")
      except:
        continue
  return embedding_matrix




In [112]:
#if want to use pre-trained embeddings make it True, if want to use trained from scratch embeddings make it False
pre_trained_embedding_matrix=create_embedding_matrix(True,my_tokenizer)
trained_embedding_matrix=create_embedding_matrix(False,my_tokenizer)

In [36]:
(label_dict.keys())

dict_keys(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC'])

In [37]:
len(label_dict.keys())

9

In [38]:
train_labels_int.shape

(14041, 124)

In [39]:
np.unique(train_labels_int)

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32)

In [40]:
train_labels_int.shape

(14041, 124)

In [41]:
#from keras.utils import to_categorical


from tensorflow.keras.utils import to_categorical

train_labels_int_onehot = [to_categorical(i, num_classes = 9) for i in train_labels_int]
val_labels_int_onehot = [to_categorical(i, num_classes = 9) for i in val_labels_int]
test_labels_int_onehot = [to_categorical(i, num_classes = 9) for i in test_labels_int]

In [42]:
train_tokens2.shape

(14041, 124)

In [43]:
np.array(train_labels_int_onehot).shape

(14041, 124, 9)

In [44]:
# Create your models and train them

In [45]:

###convert labels back to string 
def int_to_label(y_pred,test_labels_int_onehot):
  temp_pred=y_pred.copy()
  temp_label=test_labels_int_onehot.copy()

  temp_pred = np.argmax(temp_pred, axis=-1)
  temp_label = np.argmax(temp_label, axis=-1)


  temp_label2=[]
  temp_pred2=[]
  ####turns int to labels for predictions and true labels
  for i in range(len(temp_label)):
    new_arr=[]
    for j in range(len(temp_label[i])):
      for label, integer in label_dict.items():
        if temp_label[i][j] == integer:
          #print(label)
          new_arr.append(label)
    #print(new_arr)
    temp_label2.append(new_arr)

  for i in range(len(temp_pred)):
    new_arr=[]
    for j in range(len(temp_pred[i])):
      for label, integer in label_dict.items():
        if temp_pred[i][j] == integer:
          #print(label)
          new_arr.append(label)
    #print(new_arr)
    temp_pred2.append(new_arr)

  return temp_pred2,temp_label2

In [46]:
def remove_padding(y_pred):
  preds=[]
  for i in range(len(y_pred)):
    preds.append(y_pred[i][:len(test_labels[i])])
  return preds

# **RANDOM EMBEDDING**

In [47]:
train_labels_int_onehot=np.array(train_labels_int_onehot)
val_labels_int_onehot=np.array(val_labels_int_onehot)
test_labels_int_onehot=np.array(test_labels_int_onehot)

# SIMPLE RNN

In [50]:


######################################################## BASE MODEL WITH RANDOM EMBEDDING ########################################################

model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 124, 128)          32384     
                                                                 
 dropout_1 (Dropout)         (None, 124, 128)          0         
                                                                 
 time_distributed_2 (TimeDis  (None, 124, 20)          2580      
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 124, 9)           189       
 tributed)                                                       
                                                                 
Total params: 2,640,393
Trainable params: 2,640,393
No

In [51]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 71s 160ms/step - loss: 0.1281 - accuracy: 0.9718 - val_loss: 0.0955 - val_accuracy: 0.9796
Epoch 2/5
439/439 [==============================] - 61s 140ms/step - loss: 0.0366 - accuracy: 0.9884 - val_loss: 0.1145 - val_accuracy: 0.9832
Epoch 3/5
439/439 [==============================] - 56s 128ms/step - loss: 0.0181 - accuracy: 0.9950 - val_loss: 0.1248 - val_accuracy: 0.9846
Epoch 4/5
439/439 [==============================] - 54s 124ms/step - loss: 0.0087 - accuracy: 0.9976 - val_loss: 0.1452 - val_accuracy: 0.9853
Epoch 5/5
108/108 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.41      0.41      1668
        MISC       0.37      0.35      0.36       702
         ORG       0.51      0.40      0.45      1661
         PER       0.31      0.19      0.23      1617

   micro avg       0.41      0.34      0.37      5648
   macro avg       0.40      0.3

In [54]:
 ####dense arttırıldı 20->40 ismple rnn 128
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))


model.add(keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 124, 128)          32384     
                                                                 
 dropout_3 (Dropout)         (None, 124, 128)          0         
                                                                 
 time_distributed_6 (TimeDis  (None, 124, 40)          5160      
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 124, 9)           369       
 tributed)                                                       
                                                                 
Total params: 2,643,153
Trainable params: 2,643,153
No

In [55]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 54s 120ms/step - loss: 0.1083 - accuracy: 0.9769 - val_loss: 0.1023 - val_accuracy: 0.9812
Epoch 2/5
439/439 [==============================] - 52s 118ms/step - loss: 0.0299 - accuracy: 0.9915 - val_loss: 0.1141 - val_accuracy: 0.9843
Epoch 3/5
439/439 [==============================] - 52s 119ms/step - loss: 0.0128 - accuracy: 0.9964 - val_loss: 0.1283 - val_accuracy: 0.9847
Epoch 4/5
439/439 [==============================] - 70s 160ms/step - loss: 0.0070 - accuracy: 0.9980 - val_loss: 0.1433 - val_accuracy: 0.9852
Epoch 5/5
108/108 [==============================] - 2s 12ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.40      0.40      1668
        MISC       0.36      0.35      0.36       702
         ORG       0.48      0.39      0.43      1661
         PER       0.32      0.19      0.24      1617

   micro avg       0.40      0.33      0.36      5648
   macro avg       0.39      0.3

In [56]:
####dense 20, simplernn 256 oldu
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

model.add(keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 124, 256)          97536     
                                                                 
 dropout_4 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_8 (TimeDis  (None, 124, 20)          5140      
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 124, 9)           189       
 tributed)                                                       
                                                                 
Total params: 2,708,105
Trainable params: 2,708,105
No

In [57]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 60s 134ms/step - loss: 0.0936 - accuracy: 0.9799 - val_loss: 0.1102 - val_accuracy: 0.9807
Epoch 2/5
439/439 [==============================] - 79s 180ms/step - loss: 0.0270 - accuracy: 0.9920 - val_loss: 0.1187 - val_accuracy: 0.9845
Epoch 3/5
439/439 [==============================] - 65s 147ms/step - loss: 0.0114 - accuracy: 0.9968 - val_loss: 0.1392 - val_accuracy: 0.9853
Epoch 4/5
439/439 [==============================] - 82s 188ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 0.1388 - val_accuracy: 0.9851
Epoch 5/5
108/108 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

         LOC       0.40      0.40      0.40      1668
        MISC       0.37      0.38      0.37       702
         ORG       0.48      0.39      0.43      1661
         PER       0.30      0.17      0.22      1617

   micro avg       0.40      0.33      0.36      5648
   macro avg       0.39      0.3

In [58]:
##dense arttırıldı 20->40 , simplernn 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

model.add(keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 124, 256)          97536     
                                                                 
 dropout_5 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_10 (TimeDi  (None, 124, 40)          10280     
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                                 
Total params: 2,713,425
Trainable params: 2,713,425
No

In [59]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 57s 129ms/step - loss: 0.1134 - accuracy: 0.9784 - val_loss: 0.1044 - val_accuracy: 0.9791
Epoch 2/5
439/439 [==============================] - 57s 129ms/step - loss: 0.0320 - accuracy: 0.9901 - val_loss: 0.1235 - val_accuracy: 0.9839
Epoch 3/5
439/439 [==============================] - 60s 136ms/step - loss: 0.0140 - accuracy: 0.9961 - val_loss: 0.1321 - val_accuracy: 0.9850
Epoch 4/5
439/439 [==============================] - 87s 197ms/step - loss: 0.0075 - accuracy: 0.9978 - val_loss: 0.1407 - val_accuracy: 0.9852
Epoch 5/5
108/108 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.39      0.40      1668
        MISC       0.37      0.37      0.37       702
         ORG       0.51      0.39      0.44      1661
         PER       0.30      0.17      0.22      1617

   micro avg       0.41      0.33      0.36      5648
   macro avg       0.40      0.3

# BILSTM

In [60]:
##dense  20, BILSTM 128 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 bidirectional (Bidirectiona  (None, 124, 256)         259072    
 l)                                                              
                                                                 
 dropout_6 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_12 (TimeDi  (None, 124, 20)          5140      
 stributed)                                                      
                                                                 
 time_distributed_13 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                      

In [61]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 72s 153ms/step - loss: 0.1056 - accuracy: 0.9802 - val_loss: 0.1045 - val_accuracy: 0.9798
Epoch 2/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0364 - accuracy: 0.9885 - val_loss: 0.1313 - val_accuracy: 0.9823
Epoch 3/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0206 - accuracy: 0.9937 - val_loss: 0.1450 - val_accuracy: 0.9844
Epoch 4/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0116 - accuracy: 0.9966 - val_loss: 0.1474 - val_accuracy: 0.9852
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.42      0.41      0.41      1668
        MISC       0.42      0.39      0.40       702
         ORG       0.53      0.42      0.47      1661
         PER       0.31      0.18      0.22      1617

   micro avg       0.43      0.34      0.38      5648
   macro avg       0.42      0.35      

In [62]:
##dense  20,->40 BILSTM 128 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 bidirectional_1 (Bidirectio  (None, 124, 256)         259072    
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_14 (TimeDi  (None, 124, 40)          10280     
 stributed)                                                      
                                                                 
 time_distributed_15 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                      

In [63]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 21ms/step - loss: 0.1136 - accuracy: 0.9781 - val_loss: 0.1060 - val_accuracy: 0.9799
Epoch 2/5
439/439 [==============================] - 9s 19ms/step - loss: 0.0343 - accuracy: 0.9893 - val_loss: 0.1278 - val_accuracy: 0.9827
Epoch 3/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0175 - accuracy: 0.9948 - val_loss: 0.1370 - val_accuracy: 0.9848
Epoch 4/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0094 - accuracy: 0.9972 - val_loss: 0.1378 - val_accuracy: 0.9853
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.42      0.42      1668
        MISC       0.44      0.40      0.42       702
         ORG       0.54      0.43      0.48      1661
         PER       0.30      0.16      0.21      1617

   micro avg       0.43      0.35      0.38      5648
   macro avg       0.42      0.35      0

In [64]:
##dense  20, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 bidirectional_2 (Bidirectio  (None, 124, 512)         780288    
 nal)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 124, 512)          0         
                                                                 
 time_distributed_16 (TimeDi  (None, 124, 20)          10260     
 stributed)                                                      
                                                                 
 time_distributed_17 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                      

In [65]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 22ms/step - loss: 0.0964 - accuracy: 0.9793 - val_loss: 0.1041 - val_accuracy: 0.9794
Epoch 2/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0357 - accuracy: 0.9887 - val_loss: 0.1282 - val_accuracy: 0.9826
Epoch 3/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0177 - accuracy: 0.9948 - val_loss: 0.1318 - val_accuracy: 0.9847
Epoch 4/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0098 - accuracy: 0.9972 - val_loss: 0.1368 - val_accuracy: 0.9852
Epoch 5/5
108/108 [==============================] - 1s 8ms/step
              precision    recall  f1-score   support

         LOC       0.44      0.40      0.42      1668
        MISC       0.40      0.39      0.39       702
         ORG       0.57      0.46      0.51      1661
         PER       0.31      0.17      0.22      1617

   micro avg       0.45      0.35      0.39      5648
   macro avg       0.43      0.36      0

In [66]:
##dense  20, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 124, input_length=max_pad))

model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 124, 124)          2605240   
                                                                 
 bidirectional_3 (Bidirectio  (None, 124, 512)         780288    
 nal)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 124, 512)          0         
                                                                 
 time_distributed_18 (TimeDi  (None, 124, 40)          20520     
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                      

In [67]:

model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 13s 22ms/step - loss: 0.0953 - accuracy: 0.9795 - val_loss: 0.1016 - val_accuracy: 0.9795
Epoch 2/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0332 - accuracy: 0.9896 - val_loss: 0.1249 - val_accuracy: 0.9832
Epoch 3/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0157 - accuracy: 0.9953 - val_loss: 0.1298 - val_accuracy: 0.9849
Epoch 4/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0087 - accuracy: 0.9975 - val_loss: 0.1476 - val_accuracy: 0.9853
Epoch 5/5
108/108 [==============================] - 2s 10ms/step
              precision    recall  f1-score   support

         LOC       0.43      0.42      0.42      1668
        MISC       0.44      0.40      0.42       702
         ORG       0.56      0.44      0.50      1661
         PER       0.30      0.17      0.21      1617

   micro avg       0.44      0.35      0.39      5648
   macro avg       0.43      0.36      

# **PRE-TRAINED EMBEDDING**

# SIMPLE RNN 

In [70]:


######################################################## BASE MODEL WITH PRE-TRAINED EMBEDDING ########################################################
#simp RNN 128 dense 20
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 124, 128)          22912     
                                                                 
 dropout_10 (Dropout)        (None, 124, 128)          0         
                                                                 
 time_distributed_20 (TimeDi  (None, 124, 20)          2580      
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                                 
Total params: 1,076,181
Trainable params: 1,076,181
N

In [71]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 56s 124ms/step - loss: 0.1124 - accuracy: 0.9730 - val_loss: 0.0956 - val_accuracy: 0.9822
Epoch 2/5
439/439 [==============================] - 51s 116ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.1087 - val_accuracy: 0.9839
Epoch 3/5
439/439 [==============================] - 50s 114ms/step - loss: 0.0239 - accuracy: 0.9932 - val_loss: 0.1160 - val_accuracy: 0.9845
Epoch 4/5
439/439 [==============================] - 52s 119ms/step - loss: 0.0183 - accuracy: 0.9948 - val_loss: 0.1255 - val_accuracy: 0.9850
Epoch 5/5
108/108 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.41      0.41      1668
        MISC       0.39      0.36      0.38       702
         ORG       0.47      0.40      0.43      1661
         PER       0.31      0.18      0.23      1617

   micro avg       0.41      0.34      0.37      5648
   macro avg       0.40      0.3

In [72]:
#simpRNN 128 dense 20->40

model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 124, 128)          22912     
                                                                 
 dropout_11 (Dropout)        (None, 124, 128)          0         
                                                                 
 time_distributed_22 (TimeDi  (None, 124, 40)          5160      
 stributed)                                                      
                                                                 
 time_distributed_23 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                                 
Total params: 1,078,941
Trainable params: 1,078,941
N

In [73]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 51s 115ms/step - loss: 0.1001 - accuracy: 0.9761 - val_loss: 0.0936 - val_accuracy: 0.9825
Epoch 2/5
439/439 [==============================] - 50s 114ms/step - loss: 0.0299 - accuracy: 0.9916 - val_loss: 0.1094 - val_accuracy: 0.9842
Epoch 3/5
439/439 [==============================] - 78s 179ms/step - loss: 0.0204 - accuracy: 0.9942 - val_loss: 0.1145 - val_accuracy: 0.9846
Epoch 4/5
439/439 [==============================] - 58s 131ms/step - loss: 0.0157 - accuracy: 0.9954 - val_loss: 0.1208 - val_accuracy: 0.9848
Epoch 5/5
108/108 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

         LOC       0.40      0.40      0.40      1668
        MISC       0.35      0.35      0.35       702
         ORG       0.45      0.41      0.43      1661
         PER       0.31      0.19      0.23      1617

   micro avg       0.39      0.34      0.36      5648
   macro avg       0.38      0.3

In [75]:
#simpRNN 256 dense 20

model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 124, 256)          78592     
                                                                 
 dropout_12 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_24 (TimeDi  (None, 124, 20)          5140      
 stributed)                                                      
                                                                 
 time_distributed_25 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                                 
Total params: 1,134,421
Trainable params: 1,134,421
N

In [76]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 57s 128ms/step - loss: 0.0974 - accuracy: 0.9784 - val_loss: 0.0993 - val_accuracy: 0.9836
Epoch 2/5
439/439 [==============================] - 58s 133ms/step - loss: 0.0258 - accuracy: 0.9929 - val_loss: 0.1076 - val_accuracy: 0.9844
Epoch 3/5
439/439 [==============================] - 57s 129ms/step - loss: 0.0182 - accuracy: 0.9948 - val_loss: 0.1133 - val_accuracy: 0.9848
Epoch 4/5
439/439 [==============================] - 56s 127ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.1221 - val_accuracy: 0.9850
Epoch 5/5
108/108 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

         LOC       0.40      0.42      0.41      1668
        MISC       0.39      0.38      0.38       702
         ORG       0.50      0.38      0.44      1661
         PER       0.31      0.19      0.24      1617

   micro avg       0.41      0.34      0.37      5648
   macro avg       0.40      0.3

In [77]:
#simpRNN 256 dense 20->40

model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 124, 256)          78592     
                                                                 
 dropout_13 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_26 (TimeDi  (None, 124, 40)          10280     
 stributed)                                                      
                                                                 
 time_distributed_27 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                                 
Total params: 1,139,741
Trainable params: 1,139,741
N

In [78]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 59s 133ms/step - loss: 0.0845 - accuracy: 0.9796 - val_loss: 0.0982 - val_accuracy: 0.9829
Epoch 2/5
439/439 [==============================] - 55s 126ms/step - loss: 0.0297 - accuracy: 0.9917 - val_loss: 0.1158 - val_accuracy: 0.9842
Epoch 3/5
439/439 [==============================] - 55s 125ms/step - loss: 0.0209 - accuracy: 0.9939 - val_loss: 0.1219 - val_accuracy: 0.9846
Epoch 4/5
439/439 [==============================] - 55s 126ms/step - loss: 0.0162 - accuracy: 0.9952 - val_loss: 0.1232 - val_accuracy: 0.9848
Epoch 5/5
108/108 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

         LOC       0.40      0.41      0.40      1668
        MISC       0.37      0.36      0.37       702
         ORG       0.51      0.40      0.45      1661
         PER       0.32      0.18      0.23      1617

   micro avg       0.41      0.33      0.37      5648
   macro avg       0.40      0.3

# BILSTM 

In [79]:
##dense  20, BILSTM 128 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_4 (Bidirectio  (None, 124, 256)         183296    
 nal)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_28 (TimeDi  (None, 124, 20)          5140      
 stributed)                                                      
                                                                 
 time_distributed_29 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                     

In [80]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 20ms/step - loss: 0.1077 - accuracy: 0.9823 - val_loss: 0.0903 - val_accuracy: 0.9827
Epoch 2/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0257 - accuracy: 0.9925 - val_loss: 0.1058 - val_accuracy: 0.9844
Epoch 3/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0164 - accuracy: 0.9951 - val_loss: 0.1146 - val_accuracy: 0.9852
Epoch 4/5
439/439 [==============================] - 8s 18ms/step - loss: 0.0117 - accuracy: 0.9965 - val_loss: 0.1231 - val_accuracy: 0.9853
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.42      0.37      0.39      1668
        MISC       0.45      0.38      0.41       702
         ORG       0.54      0.48      0.51      1661
         PER       0.32      0.18      0.23      1617

   micro avg       0.44      0.35      0.39      5648
   macro avg       0.43      0.35      0

In [81]:
##dense  20->40, BILSTM 128 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_5 (Bidirectio  (None, 124, 256)         183296    
 nal)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_30 (TimeDi  (None, 124, 40)          10280     
 stributed)                                                      
                                                                 
 time_distributed_31 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                     

In [82]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 20ms/step - loss: 0.0989 - accuracy: 0.9837 - val_loss: 0.0983 - val_accuracy: 0.9828
Epoch 2/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0251 - accuracy: 0.9927 - val_loss: 0.1128 - val_accuracy: 0.9846
Epoch 3/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0153 - accuracy: 0.9955 - val_loss: 0.1220 - val_accuracy: 0.9852
Epoch 4/5
439/439 [==============================] - 8s 18ms/step - loss: 0.0105 - accuracy: 0.9969 - val_loss: 0.1326 - val_accuracy: 0.9854
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.44      0.42      0.43      1668
        MISC       0.46      0.42      0.44       702
         ORG       0.60      0.45      0.52      1661
         PER       0.30      0.18      0.23      1617

   micro avg       0.46      0.36      0.40      5648
   macro avg       0.45      0.37      0

In [83]:
##dense  20-, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_6 (Bidirectio  (None, 124, 512)         628736    
 nal)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 124, 512)          0         
                                                                 
 time_distributed_32 (TimeDi  (None, 124, 20)          10260     
 stributed)                                                      
                                                                 
 time_distributed_33 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                     

In [84]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 13s 22ms/step - loss: 0.0814 - accuracy: 0.9844 - val_loss: 0.0930 - val_accuracy: 0.9826
Epoch 2/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0229 - accuracy: 0.9932 - val_loss: 0.1099 - val_accuracy: 0.9845
Epoch 3/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0139 - accuracy: 0.9959 - val_loss: 0.1176 - val_accuracy: 0.9852
Epoch 4/5
439/439 [==============================] - 9s 21ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.1226 - val_accuracy: 0.9856
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.43      0.41      0.42      1668
        MISC       0.47      0.41      0.44       702
         ORG       0.59      0.47      0.53      1661
         PER       0.30      0.18      0.22      1617

   micro avg       0.45      0.36      0.40      5648
   macro avg       0.45      0.37      0

In [85]:
##dense  20->40, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_7 (Bidirectio  (None, 124, 512)         628736    
 nal)                                                            
                                                                 
 dropout_17 (Dropout)        (None, 124, 512)          0         
                                                                 
 time_distributed_34 (TimeDi  (None, 124, 40)          20520     
 stributed)                                                      
                                                                 
 time_distributed_35 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                     

In [86]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 13s 22ms/step - loss: 0.0793 - accuracy: 0.9846 - val_loss: 0.0962 - val_accuracy: 0.9828
Epoch 2/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0220 - accuracy: 0.9935 - val_loss: 0.1077 - val_accuracy: 0.9845
Epoch 3/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0137 - accuracy: 0.9959 - val_loss: 0.1169 - val_accuracy: 0.9851
Epoch 4/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0096 - accuracy: 0.9971 - val_loss: 0.1242 - val_accuracy: 0.9854
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.44      0.40      0.42      1668
        MISC       0.47      0.41      0.44       702
         ORG       0.58      0.47      0.52      1661
         PER       0.30      0.18      0.22      1617

   micro avg       0.45      0.36      0.40      5648
   macro avg       0.45      0.36      0

# **TRAINED EMBEDDING**

# SIMPLE RNN

In [119]:


######################################################## BASE MODEL WITH TRAINED EMBEDDING ########################################################
#simp RNN 128 dense 20
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 124, 128)          22912     
                                                                 
 dropout_20 (Dropout)        (None, 124, 128)          0         
                                                                 
 time_distributed_40 (TimeDi  (None, 124, 20)          2580      
 stributed)                                                      
                                                                 
 time_distributed_41 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                                 
Total params: 1,076,181
Trainable params: 1,076,181
N

In [120]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 74s 165ms/step - loss: 0.1050 - accuracy: 0.9750 - val_loss: 0.1093 - val_accuracy: 0.9804
Epoch 2/5
439/439 [==============================] - 52s 117ms/step - loss: 0.0395 - accuracy: 0.9884 - val_loss: 0.1201 - val_accuracy: 0.9826
Epoch 3/5
439/439 [==============================] - 52s 118ms/step - loss: 0.0229 - accuracy: 0.9937 - val_loss: 0.1328 - val_accuracy: 0.9844
Epoch 4/5
439/439 [==============================] - 52s 118ms/step - loss: 0.0148 - accuracy: 0.9960 - val_loss: 0.1369 - val_accuracy: 0.9846
Epoch 5/5
108/108 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

         LOC       0.40      0.38      0.39      1668
        MISC       0.37      0.34      0.36       702
         ORG       0.52      0.39      0.45      1661
         PER       0.27      0.16      0.20      1617

   micro avg       0.40      0.31      0.35      5648
   macro avg       0.39      0.3

In [121]:

#simp RNN 128 dense 20->40
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 124, 128)          22912     
                                                                 
 dropout_21 (Dropout)        (None, 124, 128)          0         
                                                                 
 time_distributed_42 (TimeDi  (None, 124, 40)          5160      
 stributed)                                                      
                                                                 
 time_distributed_43 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                                 
Total params: 1,078,941
Trainable params: 1,078,941
N

In [122]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 53s 119ms/step - loss: 0.0938 - accuracy: 0.9768 - val_loss: 0.1043 - val_accuracy: 0.9804
Epoch 2/5
439/439 [==============================] - 52s 119ms/step - loss: 0.0367 - accuracy: 0.9887 - val_loss: 0.1258 - val_accuracy: 0.9824
Epoch 3/5
439/439 [==============================] - 52s 119ms/step - loss: 0.0218 - accuracy: 0.9937 - val_loss: 0.1311 - val_accuracy: 0.9846
Epoch 4/5
439/439 [==============================] - 52s 118ms/step - loss: 0.0140 - accuracy: 0.9961 - val_loss: 0.1329 - val_accuracy: 0.9846
Epoch 5/5
108/108 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.40      0.41      1668
        MISC       0.38      0.35      0.37       702
         ORG       0.50      0.38      0.43      1661
         PER       0.29      0.17      0.22      1617

   micro avg       0.41      0.32      0.36      5648
   macro avg       0.40      0.3

In [123]:

#simp RNN 256 dense 20
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 124, 256)          78592     
                                                                 
 dropout_22 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_44 (TimeDi  (None, 124, 20)          5140      
 stributed)                                                      
                                                                 
 time_distributed_45 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                                 
Total params: 1,134,421
Trainable params: 1,134,421
N

In [124]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 62s 140ms/step - loss: 0.0970 - accuracy: 0.9761 - val_loss: 0.1132 - val_accuracy: 0.9806
Epoch 2/5
439/439 [==============================] - 56s 128ms/step - loss: 0.0359 - accuracy: 0.9896 - val_loss: 0.1209 - val_accuracy: 0.9835
Epoch 3/5
439/439 [==============================] - 56s 129ms/step - loss: 0.0194 - accuracy: 0.9947 - val_loss: 0.1365 - val_accuracy: 0.9848
Epoch 4/5
439/439 [==============================] - 58s 133ms/step - loss: 0.0123 - accuracy: 0.9965 - val_loss: 0.1370 - val_accuracy: 0.9849
Epoch 5/5
108/108 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

         LOC       0.40      0.41      0.41      1668
        MISC       0.37      0.35      0.36       702
         ORG       0.49      0.38      0.43      1661
         PER       0.29      0.17      0.21      1617

   micro avg       0.40      0.33      0.36      5648
   macro avg       0.39      0.3

In [125]:

#simp RNN 256 dense 20->40
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

#model.add(tf.keras.layers.Conv1D (128, 5, padding = 'same', activation='relu'))

model.add(keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 124, 256)          78592     
                                                                 
 dropout_23 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_46 (TimeDi  (None, 124, 40)          10280     
 stributed)                                                      
                                                                 
 time_distributed_47 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                                 
Total params: 1,139,741
Trainable params: 1,139,741
N

In [126]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 57s 129ms/step - loss: 0.0917 - accuracy: 0.9758 - val_loss: 0.1193 - val_accuracy: 0.9810
Epoch 2/5
439/439 [==============================] - 58s 133ms/step - loss: 0.0357 - accuracy: 0.9892 - val_loss: 0.1208 - val_accuracy: 0.9829
Epoch 3/5
439/439 [==============================] - 57s 129ms/step - loss: 0.0193 - accuracy: 0.9946 - val_loss: 0.1330 - val_accuracy: 0.9848
Epoch 4/5
439/439 [==============================] - 56s 129ms/step - loss: 0.0124 - accuracy: 0.9965 - val_loss: 0.1340 - val_accuracy: 0.9849
Epoch 5/5
108/108 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.40      0.41      1668
        MISC       0.41      0.38      0.39       702
         ORG       0.46      0.39      0.42      1661
         PER       0.29      0.17      0.21      1617

   micro avg       0.40      0.33      0.36      5648
   macro avg       0.39      0.3

# BILSTM

In [127]:
##dense  20, BILSTM 128 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_8 (Bidirectio  (None, 124, 256)         183296    
 nal)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_48 (TimeDi  (None, 124, 20)          5140      
 stributed)                                                      
                                                                 
 time_distributed_49 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                     

In [128]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 21ms/step - loss: 0.1162 - accuracy: 0.9793 - val_loss: 0.1011 - val_accuracy: 0.9809
Epoch 2/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0378 - accuracy: 0.9887 - val_loss: 0.1206 - val_accuracy: 0.9823
Epoch 3/5
439/439 [==============================] - 9s 21ms/step - loss: 0.0228 - accuracy: 0.9929 - val_loss: 0.1395 - val_accuracy: 0.9838
Epoch 4/5
439/439 [==============================] - 8s 18ms/step - loss: 0.0145 - accuracy: 0.9958 - val_loss: 0.1453 - val_accuracy: 0.9850
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.40      0.40      1668
        MISC       0.45      0.40      0.42       702
         ORG       0.51      0.44      0.47      1661
         PER       0.31      0.16      0.21      1617

   micro avg       0.43      0.34      0.38      5648
   macro avg       0.42      0.35      0

In [129]:
##dense  20->40, BILSTM 128 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_29 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_9 (Bidirectio  (None, 124, 256)         183296    
 nal)                                                            
                                                                 
 dropout_25 (Dropout)        (None, 124, 256)          0         
                                                                 
 time_distributed_50 (TimeDi  (None, 124, 40)          10280     
 stributed)                                                      
                                                                 
 time_distributed_51 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                     

In [130]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 20ms/step - loss: 0.1022 - accuracy: 0.9812 - val_loss: 0.1047 - val_accuracy: 0.9810
Epoch 2/5
439/439 [==============================] - 8s 18ms/step - loss: 0.0329 - accuracy: 0.9899 - val_loss: 0.1258 - val_accuracy: 0.9830
Epoch 3/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0187 - accuracy: 0.9945 - val_loss: 0.1328 - val_accuracy: 0.9845
Epoch 4/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0112 - accuracy: 0.9968 - val_loss: 0.1396 - val_accuracy: 0.9851
Epoch 5/5
108/108 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

         LOC       0.42      0.41      0.42      1668
        MISC       0.47      0.41      0.44       702
         ORG       0.59      0.44      0.50      1661
         PER       0.29      0.17      0.22      1617

   micro avg       0.45      0.35      0.39      5648
   macro avg       0.44      0.36      0

In [131]:
##dense  20, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(20, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_10 (Bidirecti  (None, 124, 512)         628736    
 onal)                                                           
                                                                 
 dropout_26 (Dropout)        (None, 124, 512)          0         
                                                                 
 time_distributed_52 (TimeDi  (None, 124, 20)          10260     
 stributed)                                                      
                                                                 
 time_distributed_53 (TimeDi  (None, 124, 9)           189       
 stributed)                                                      
                                                     

In [132]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 14s 25ms/step - loss: 0.0842 - accuracy: 0.9831 - val_loss: 0.1146 - val_accuracy: 0.9813
Epoch 2/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0296 - accuracy: 0.9909 - val_loss: 0.1330 - val_accuracy: 0.9831
Epoch 3/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0180 - accuracy: 0.9946 - val_loss: 0.1287 - val_accuracy: 0.9843
Epoch 4/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0117 - accuracy: 0.9967 - val_loss: 0.1367 - val_accuracy: 0.9851
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.42      0.42      1668
        MISC       0.40      0.42      0.41       702
         ORG       0.55      0.45      0.49      1661
         PER       0.32      0.15      0.20      1617

   micro avg       0.44      0.35      0.39      5648
   macro avg       0.42      0.36      0

In [133]:
##dense  20->40, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_31 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_11 (Bidirecti  (None, 124, 512)         628736    
 onal)                                                           
                                                                 
 dropout_27 (Dropout)        (None, 124, 512)          0         
                                                                 
 time_distributed_54 (TimeDi  (None, 124, 40)          20520     
 stributed)                                                      
                                                                 
 time_distributed_55 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                     

In [134]:


model.fit(train_tokens2, train_labels_int_onehot, epochs=5, verbose=1,validation_data=(val_tokens2, val_labels_int_onehot))
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/5
439/439 [==============================] - 12s 21ms/step - loss: 0.0921 - accuracy: 0.9814 - val_loss: 0.1062 - val_accuracy: 0.9814
Epoch 2/5
439/439 [==============================] - 8s 19ms/step - loss: 0.0311 - accuracy: 0.9905 - val_loss: 0.1311 - val_accuracy: 0.9834
Epoch 3/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0175 - accuracy: 0.9949 - val_loss: 0.1407 - val_accuracy: 0.9848
Epoch 4/5
439/439 [==============================] - 9s 20ms/step - loss: 0.0103 - accuracy: 0.9972 - val_loss: 0.1446 - val_accuracy: 0.9852
Epoch 5/5
108/108 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

         LOC       0.41      0.42      0.41      1668
        MISC       0.43      0.42      0.43       702
         ORG       0.59      0.42      0.49      1661
         PER       0.30      0.17      0.22      1617

   micro avg       0.44      0.35      0.39      5648
   macro avg       0.43      0.36      0

# Best Model Trained with Train + Val

In [135]:
train_tokens2_temp=train_tokens2.copy()
val_tokens2_temp=val_tokens2.copy()

train_labels_int_onehot_temp=train_labels_int_onehot.copy()
val_labels_int_onehot_temp=val_labels_int_onehot.copy()

train_and_val_tokens=np.concatenate((train_tokens2_temp, val_tokens2_temp), axis=0)

train_and_val_labels=np.concatenate((train_labels_int_onehot_temp, val_labels_int_onehot_temp), axis=0)

In [143]:
##dense  20->40, BILSTM 256 
model = Sequential()

# Add Embedding layer
model.add(Embedding(vocab_size, 50 ,input_length=max_pad,weights=[pre_trained_embedding_matrix],trainable=True))

model.add(Bidirectional(LSTM(512,return_sequences=True)))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(40, activation="relu")))
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(len(label_dict.keys()), activation="softmax")))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_35 (Embedding)    (None, 124, 50)           1050500   
                                                                 
 bidirectional_15 (Bidirecti  (None, 124, 1024)        2306048   
 onal)                                                           
                                                                 
 dropout_31 (Dropout)        (None, 124, 1024)         0         
                                                                 
 time_distributed_62 (TimeDi  (None, 124, 40)          41000     
 stributed)                                                      
                                                                 
 time_distributed_63 (TimeDi  (None, 124, 9)           369       
 stributed)                                                      
                                                     

In [144]:


model.fit(train_and_val_tokens, train_and_val_labels, epochs=20, verbose=1)
# evaluate the model

y_pred = model.predict(test_tokens2, verbose=1)
pred,true_labels=int_to_label(y_pred,test_labels_int_onehot)##convert integers back to labels
preds=remove_padding(pred)### remove padding 

from seqeval.metrics import classification_report
print(classification_report(test_labels, preds))

###https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

Epoch 1/20
541/541 [==============================] - 22s 34ms/step - loss: 0.0692 - accuracy: 0.9850
Epoch 2/20
541/541 [==============================] - 18s 34ms/step - loss: 0.0280 - accuracy: 0.9928
Epoch 3/20
541/541 [==============================] - 19s 34ms/step - loss: 0.0210 - accuracy: 0.9946
Epoch 4/20
541/541 [==============================] - 19s 34ms/step - loss: 0.0170 - accuracy: 0.9956
Epoch 5/20
541/541 [==============================] - 19s 35ms/step - loss: 0.0145 - accuracy: 0.9961
Epoch 6/20
541/541 [==============================] - 19s 35ms/step - loss: 0.0120 - accuracy: 0.9966
Epoch 7/20
541/541 [==============================] - 19s 35ms/step - loss: 0.0103 - accuracy: 0.9970
Epoch 8/20
541/541 [==============================] - 19s 35ms/step - loss: 0.0084 - accuracy: 0.9974
Epoch 9/20
541/541 [==============================] - 19s 35ms/step - loss: 0.0074 - accuracy: 0.9976
Epoch 10/20
541/541 [==============================] - 19s 35ms/step - loss: 0.005

# **MY REPORT**

## **CRF** 
First I started by reading the data,read it line by line and extracting the word, pos tag, chunk tag and the ner tag. I keep each sentence as different arrays in a big array. I also eliminate the docstart lines since we dont need them. Use this function for creating train val and test data. Later I create the gazetteer using the wikipedia pages. According to my knowledge we need to extract all the href link titles since all of them are kind of a special entity.  I observed that href titles are located between the special charcters of "&gt ; and /a&gt therefore I created a regex to extract the href strings then added them all to a list one by one. 

CRF Features:

I used https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html to extract the crf features. For neighbor features I added 2 if cases to check the previous and next word existance and edit the neighbor features accordingly. I also defined functions to extract word shape, and used word shape function to create short word shape. I applied token2features to all sentences in the dataset as well as created label lists for train val and test data.

After having the sentences and labels data ready,  I performed Grid search for CRF I, best parameters were {'algorithm': 'lbfgs', 'all_possible_transitions': True, 'max_iterations': 100} 
By using them I predict the test data and obtained 0.82 macro F1 score.


Later I trained different crf models with adding each features one by one cumulatively.  I did this by taking sub-sample of feature dictionary   every time for new features. Every time I predict the validation dataset and saved each metric for later visualization. To visualize the affect of each feature on the dataset I created a dataframe with my saved arrays.

###Analysis:

As expected the best resulting feature sample is the one that contains every feature. When testing on test data using all the features I obtained 0.81 macro F1 and 0.96 accuracy score. The F1 score starts with 0.60 by just using stem as a feature, the greatest jump in f1 score is done by the pos_tag it increases the score by 0.13 but we cannot observe other features that has this much affect they generally increase the score by 0.03 etc. The seocnd most affcetive feature was adding the nieghbour to feature set.

## **RNN**

I trained in total 24 models, grouped them in 3 ; Random embedding, pre-trained embedding and trained from scratch embedding. All models are trained for 5 epochs.

RANDOM:

- I tried Simple RNN layer with 128 and Dense layer as 20 at initial. It gave me -

- Simple Rnn with 128 units and dense layer with 40 

- Simple Rnn with 256 units and dense layer with 20 

- Simple Rnn with 256 units and dense layer with 40 

- BiLstm with 128 and dense 20
- BiLstm with 128 and dense 40
- BiLstm with 256 and dense 20
- BiLstm with 256 and dense 40

Changing Simple RNN parameters did not affect the model much, the macro f1 generally resulted in 0.36 and it took longer time to train than Lstm layer.
I observed that even though the train loss became quickly less the validation loss started increasing at every epoch this showed overfitting therefore I also added dropout layer with 0.5 probability to overcomet this however it did not change the results much.

PRETRAINED EMBEDDING:

The above combinations are also trained but this time with an embedding layer from gensim api. I choose glpve twitter data and embedding size of 50.
As expected pre-trained model performed slightly better than random embeddings I observed the highest 0.4  macro F1 with BIlstm layers on test set.


TRAINED EMBEDDING: 

The above combinations are also trained but this time with an embedding layer trained from scratch with our own train dataset using Word2Vec again I choose size as 50. Trained embedding matrix did not performed as well as pre-trained it is similar to random embedding Simple Rnn fluctuates around 0.36 and Bilstm around 0.39


I also tried using LSTM layer which is not bidirectional like bilstm but it did not perform as good as bilstm therefore I did not include it in models. Finally I trained the best performed model with using both train and validation data as train data, when I trained it for longer epochs like 10 and 20 I observed better Macro F1 scores, the highest achieved is 0.47 with 512 bilstm layer and 20 epochs. Abover combinations would probably perform a little bit better if they were trained for longer.

All in all, I obtained way better results when using CRF however it is computaitonally more expensive and time consuming to extract all the features.




In [146]:
%%shell
jupyter nbconvert --to html /content/Project_2_Notebook.ipynb

[NbConvertApp] Converting notebook /content/Project_2_Notebook.ipynb to html
/usr/local/lib/python3.7/dist-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.
  mimetypes=output.keys())
[NbConvertApp] Writing 639668 bytes to /content/Project_2_Notebook.html
